In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
from glob import glob

In [ ]:
dfs = []
for fn in glob('learningsys-2018-gpu-mux/**/*.pq', recursive=True):
    df = pd.read_parquet(fn)
    _, _, approach, model, proc, i  = fn.split("/")
    df['approach'] = approach
    df['model'] = model
    df['n_procs'] = int(proc)
    df['proc'] = int(i.split('.')[0])
    dfs.append(df)

In [ ]:
df = pd.concat(dfs)

In [ ]:
df.head(2)

In [ ]:
anomoly = df.groupby(
    ['approach', 'model', 'n_procs', 'proc']
)[['duration_ms']].mean().reset_index().groupby(
    ['approach', 'model', 'n_procs'])[['duration_ms']].apply(lambda s: s.max() - s.min()).reset_index()

In [ ]:
anomoly = anomoly[anomoly['duration_ms']>0.1]

In [ ]:
anomoly = anomoly[anomoly['approach'] == 'mps']

In [ ]:
anomoly

In [ ]:
sns.set(style='whitegrid', palette='muted', font='serif')

In [ ]:
grouped = res152_5_procs.groupby('proc')[['duration_ms']].rolling(10).mean().dropna().reset_index()

In [ ]:
grouped.head()

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
sns.pointplot(x='n_procs', y='duration_ms', hue='model', data=anomoly)
plt.axhline(0.1, label='0.1 lower bound')
plt.legend()
plt.xlabel('Number of Replicas')
plt.ylabel('Latency (ms)')
plt.title('Max difference between processes')

plt.subplot(1,2,2)
res152_5_procs = df[(df['model'] == 'res152') & (df['n_procs'] == 5) & (df['approach'] == 'mps')][['duration_ms', 'proc', 'query_id']]
sns.lineplot(x='level_1', y='duration_ms', hue='proc', data=grouped)
plt.xlabel('Query IDs')
plt.ylabel('Latency (ms)')
plt.title('Example: Res152, Mux, 5 Processes')

plt.savefig('anomoly.png', dpi=300)

In [ ]:
for _, row in anomoly.iterrows():
    print(f"{row['approach']}-{row['model']}-{row['n_procs']}")

In [ ]:
len(anomoly)